# Clone UAP

In [ ]:
!git clone https://github.com/theFool32/UAP_retrieval.git

Cloning into 'UAP_retrieval'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 72 (delta 0), reused 0 (delta 0), pack-reused 69
Unpacking objects: 100% (72/72), done.


In [ ]:
%cd /content/UAP_retrieval

/content/UAP_retrieval


# Download Oxford5k or Paris6k Images

In [ ]:
%cd /content
!wget https://www.robots.ox.ac.uk/~vgg/data/oxbuildings/oxbuild_images-v1.tgz
# !wget https://www.robots.ox.ac.uk/~vgg/data/parisbuildings/paris_1-v1.tgz
# !wget https://www.robots.ox.ac.uk/~vgg/data/parisbuildings/paris_2-v1.tgz

/content
--2022-07-29 23:33:56--  https://www.robots.ox.ac.uk/~vgg/data/oxbuildings/oxbuild_images-v1.tgz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1938238004 (1.8G) [application/x-gzip]
Saving to: ‘oxbuild_images-v1.tgz’

oxbuild_images-v1.t 100%[===================>]   1.80G  20.8MB/s    in 87s     

2022-07-29 23:35:23 (21.2 MB/s) - ‘oxbuild_images-v1.tgz’ saved [1938238004/1938238004]



In [ ]:
!mkdir /content/UAP_retrieval/cirtorch/data

In [ ]:
%cd UAP_retrieval/

/content/UAP_retrieval


# Mount Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/models.zip /content/models.zip
!unzip /content/models.zip

Archive:  /content/models.zip
  inflating: models/retrievalSfM120k-alexnet-gem.pth  
  inflating: models/retrievalSfM120k-alexnet-mac.pth  
  inflating: models/retrievalSfM120k-resnet101-gem.pth  
  inflating: models/retrievalSfM120k-resnet101-mac.pth  
  inflating: models/retrievalSfM120k-vgg16-gem.pth  
  inflating: models/retrievalSfM120k-vgg16-mac.pth  


# Download Ground Truth Files

In [ ]:
%cd /content/UAP_retrieval/
!wget http://cmp.felk.cvut.cz/cnnimageretrieval/data/test/oxford5k/gnd_oxford5k.pkl 
# !wget http://cmp.felk.cvut.cz/cnnimageretrieval/data/test/paris6k/gnd_paris6k.pkl 

/content/UAP_retrieval
--2022-07-29 23:36:21--  http://cmp.felk.cvut.cz/cnnimageretrieval/data/test/oxford5k/gnd_oxford5k.pkl
Resolving cmp.felk.cvut.cz (cmp.felk.cvut.cz)... 147.32.84.2
Connecting to cmp.felk.cvut.cz (cmp.felk.cvut.cz)|147.32.84.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 141140 (138K)
Saving to: ‘gnd_oxford5k.pkl’

gnd_oxford5k.pkl    100%[===================>] 137.83K   305KB/s    in 0.5s    

2022-07-29 23:36:22 (305 KB/s) - ‘gnd_oxford5k.pkl’ saved [141140/141140]



# Some Set-Up for Images

In [ ]:
!mkdir /content/UAP_retrieval/data
!mkdir /content/UAP_retrieval/data/test/
!mkdir /content/UAP_retrieval/data/test/oxford5k/
# !mkdir /content/UAP_retrieval/data/test/paris6k/
!cp /content/UAP_retrieval/gnd_oxford5k.pkl /content/UAP_retrieval/data/test/oxford5k/gnd_oxford5k.pkl
# !cp /content/UAP_retrieval/gnd_paris6k.pkl /content/UAP_retrieval/data/test/oxford5k/gnd_paris6k.pkl

In [ ]:
!mkdir /content/UAP_retrieval/data/test/oxford5k/jpg
# !mkdir /content/UAP_retrieval/data/test/paris6k/jpg

In [ ]:
%cd /content/UAP_retrieval/data/test/oxford5k/jpg
# %cd /content/UAP_retrieval/data/test/paris6k/jpg

/content/UAP_retrieval/data/test/oxford5k/jpg


In [ ]:
!tar -xf /content/oxbuild_images-v1.tgz
# !tar -xf /content/paris_1-v1.tgz
# !tar -xf /content/paris_2-v1.tgz

In [ ]:
%cd /content/UAP_retrieval/

/content/UAP_retrieval


# For Computing mAP and Success Rate

In [ ]:
def compute_ap(ranks, nres):
    """
    Computes average precision for given ranked indexes.
    
    Arguments
    ---------
    ranks : zerro-based ranks of positive images
    nres  : number of positive images
    
    Returns
    -------
    ap    : average precision
    """

    # number of images ranked by the system
    nimgranks = len(ranks)

    # accumulate trapezoids in PR-plot
    ap = 0

    recall_step = 1. / nres

    for j in np.arange(nimgranks):
        rank = ranks[j]

        if rank == 0:
            precision_0 = 1.
        else:
            precision_0 = float(j) / rank

        precision_1 = float(j + 1) / (rank + 1)

        ap += (precision_0 + precision_1) * recall_step / 2.

    return ap

def compute_map(ranks, gnd, kappas=[]):
    """
    Computes the mAP for a given set of returned results.

         Usage: 
           map = compute_map (ranks, gnd) 
                 computes mean average precsion (map) only
        
           map, aps, pr, prs = compute_map (ranks, gnd, kappas) 
                 computes mean average precision (map), average precision (aps) for each query
                 computes mean precision at kappas (pr), precision at kappas (prs) for each query
        
         Notes:
         1) ranks starts from 0, ranks.shape = db_size X #queries
         2) The junk results (e.g., the query itself) should be declared in the gnd stuct array
         3) If there are no positive images for some query, that query is excluded from the evaluation
    """

    map = 0.
    nq = len(gnd) # number of queries
    aps = np.zeros(nq)
    pr = np.zeros(len(kappas))
    prs = np.zeros((nq, len(kappas)))
    nempty = 0

    for i in np.arange(nq):
        qgnd = np.array(gnd[i]['ok'])

        # no positive images, skip from the average
        if qgnd.shape[0] == 0:
            aps[i] = float('nan')
            prs[i, :] = float('nan')
            nempty += 1
            continue

        try:
            qgndj = np.array(gnd[i]['junk'])
        except:
            qgndj = np.empty(0)

        # sorted positions of positive and junk images (0 based)
        pos  = np.arange(ranks.shape[0])[np.in1d(ranks[:,i], qgnd)]
        junk = np.arange(ranks.shape[0])[np.in1d(ranks[:,i], qgndj)]

        k = 0;
        ij = 0;
        if len(junk):
            # decrease positions of positives based on the number of
            # junk images appearing before them
            ip = 0
            while (ip < len(pos)):
                while (ij < len(junk) and pos[ip] > junk[ij]):
                    k += 1
                    ij += 1
                pos[ip] = pos[ip] - k
                ip += 1

        # compute ap
        ap = compute_ap(pos, len(qgnd))
        map = map + ap
        aps[i] = ap

        # compute precision @ k
        pos += 1 # get it to 1-based
        for j in np.arange(len(kappas)):
            kq = min(max(pos), kappas[j]); 
            prs[i, j] = (pos <= kq).sum() / kq
        pr = pr + prs[i, :]

    map = map / (nq - nempty)
    pr = pr / (nq - nempty)

    return map, aps, pr, prs


def compute_map_and_print(dataset, ranks, gnd, kappas=[1, 5, 10]):
  
    # old evaluation protocol
    if dataset.startswith('oxford5k') or dataset.startswith('paris6k'):
        map, aps, _, _ = compute_map(ranks, gnd)
        print('>> {}: mAP {:.2f}'.format(dataset, np.around(map*100, decimals=2)))
        return [np.around(map*100, decimals=2)]

    # new evaluation protocol
    elif dataset.startswith('roxford5k') or dataset.startswith('rparis6k'):
        
        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['easy']])
            g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['hard']])
            gnd_t.append(g)
        mapE, apsE, mprE, prsE = compute_map(ranks, gnd_t, kappas)

        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
            g['junk'] = np.concatenate([gnd[i]['junk']])
            gnd_t.append(g)
        mapM, apsM, mprM, prsM = compute_map(ranks, gnd_t, kappas)

        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['hard']])
            g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
            gnd_t.append(g)
        mapH, apsH, mprH, prsH = compute_map(ranks, gnd_t, kappas)

        print('>> {}: mAP E: {}, M: {}, H: {}'.format(dataset, np.around(mapE*100, decimals=2), np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
        print('>> {}: mP@k{} E: {}, M: {}, H: {}'.format(dataset, kappas, np.around(mprE*100, decimals=2), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))
        return [np.around(v*100, decimals=2) for v in [mapE, mapM, mapH, mprE[-1], mprM[-1], mprH[-1]]]

def compute_attack_success_rate(clean_ranks, noisy_ranks):
  print(clean_ranks.shape)
  print(noisy_ranks.shape)
  # print(clean_ranks)
  num_successful = 0
  total_querys = 0
  for query in range(clean_ranks.shape[1]):
    clean_rank = clean_ranks[:10, query]
    noisy_rank = noisy_ranks[:10, query]

    print("Clean: " , clean_rank)
    print("Noisy: " , noisy_rank)
    intersect = np.intersect1d(clean_rank, noisy_rank)
    print(intersect.size)
    if (intersect.size == 0):
      num_successful += 1
    
    total_querys += 1
  print("Attack Success Rate" , num_successful / total_querys * 100)

  #   print("Clean Size: ", clean_rank.size())
  #   print("Noisy Size: ", noisy_rank.size())



# Code for Extracting Vectors from Images using Feature Extractor

In [ ]:
def extract_vectors(net, images, image_size, transform, bbxs=None, ms=[1], msp=1, print_freq=10):
    # moving network to gpu and eval mode
    net.cuda()
    net.eval()

    # creating dataset loader
    loader = torch.utils.data.DataLoader(
        ImagesFromList(root='', images=images, imsize=image_size, bbxs=bbxs, transform=transform),
        batch_size=1, shuffle=False, num_workers=4, pin_memory=True
    )
    # extracting vectors
    vecs = torch.zeros(net.meta['outputdim'], len(images))
    for i, input in enumerate(loader):
        input_var = Variable(input.cuda())
        if len(ms) == 1:
            vecs[:, i] = extract_ss(net, input_var)
        else:
            vecs[:, i] = extract_ms(net, input_var, ms, msp)

        if (i+1) % print_freq == 0 or (i+1) == len(images):
            print('\r>>>> {}/{} done...'.format((i+1), len(images)), end='')
    print('')
    return vecs


def extract_ss(net, input_var):
    return net(input_var).cpu().data.squeeze()


def extract_ms(net, input_var, ms, msp):
    
    print()
    v = torch.zeros(net.meta['outputdim'])
    
    for s in ms: 
        if s == 1:
            input_var_t = input_var.clone()
        else:    
            size = (int(input_var.size(-2) * s), int(input_var.size(-1) * s))
            input_var_t = nn.functional.upsample(input_var, size=size, mode='bilinear')
        v += net(input_var_t).pow(msp).cpu().data.squeeze()
        
    v /= len(ms)
    v = v.pow(1./msp)
    v /= v.norm()

    return v

# Dataset that returns image - 10 image pairs according to descriptor similarity

In [ ]:
import argparse
import math
import os
import pdb
import pickle
import random
import shutil
import time
from pprint import pprint

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

from cirtorch.datasets.datahelpers import cid2filename, collate_tuples
from cirtorch.datasets.testdataset import configdataset
from cirtorch.datasets.traindataset import TuplesDataset
from cirtorch.datasets.genericdataset import ImagesFromList
from cirtorch.examples.attack.myutil.baseline import result as baseline_result
from cirtorch.examples.attack.myutil.mi_sgd import (MI_SGD, SIGN_AdaBound,
                                                    SIGN_Adam)
from cirtorch.examples.attack.myutil.sfm_dataset import SfMDataset
from cirtorch.examples.attack.myutil.triplet_dataset import MyTripletDataset
from cirtorch.examples.attack.myutil.utils import (MultiLoss, bcolors,
                                                   do_whiten, idcg, inv_gfr,
                                                   one_hot, rescale_check)
from cirtorch.layers.loss import ContrastiveLoss
from cirtorch.networks.imageretrievalnet import extract_vectors, init_network
from cirtorch.utils.download import download_test, download_train
# from cirtorch.utils.evaluate import compute_map_and_print
from torch.autograd import Variable
from cirtorch.utils.general import get_data_root, htime
from cirtorch.utils.whiten import whitenapply, whitenlearn



class BlackBoxAttackDataset(data.Dataset):
  def __init__(self, model, imsize=None,
        numreturnedimgs=10,
        datasetname="oxford5k", vecs_lw=None, qvecs_lw=None, transforms2=None):
    
    self.img_size = imsize
    self.cfg = configdataset(datasetname, os.path.join(get_data_root(), "test"))
    self.dbimages = [self.cfg["im_fname"](self.cfg, i) for i in range(self.cfg["n"])]
    self.qimages = [self.cfg["qim_fname"](self.cfg, i) for i in range(self.cfg["nq"])]
    self.bbxs = [tuple(self.cfg["gnd"][i]["bbx"]) for i in range(self.cfg["nq"])]
    # load retrieval model
    
    self.model = model
    
    self.model.eval()
    normalize = transforms.Normalize(mean=self.model.meta["mean"], std=self.model.meta["std"])

    self.transform = transforms.Compose([transforms.ToTensor(), normalize])
    self.numreturnedimgs = numreturnedimgs


    if "Lw" in self.model.meta:
      Lw = self.model.meta["Lw"]["retrieval-SfM-120k"]["ss"]
    else:
      Lw = None
    self.qvecs = extract_vectors(self.model, self.qimages, self.img_size, self.transform, self.bbxs)

    self.qvecs = self.qvecs.numpy()

    self.vecs_lw = vecs_lw
    self.qvecs_lw = whitenapply(self.qvecs, Lw["m"], Lw["P"])

  def __len__(self):
    return len(self.qimages)

  def __getitem__(self, idx):
    queryimgs, retrievedresults = self.retrieve_relevant_images_from_lists(idx, self.model, self.vecs_lw, self.qvecs_lw[:, idx], [self.qimages[idx]], self.dbimages, self.img_size, self.numreturnedimgs, self.transform)
    return queryimgs, retrievedresults

  def retrieve_relevant_images_from_lists(self, idx, net, vecs_lw, qvecs_lw, qimages, images, image_size, num_returned_images, transform, bbxs=None, ms=[1], msp=1, print_freq=10):


    # Take the Dot Product of the Query Features and Database Features to get a Num_Features x Num_Queries Matrix of Similarities
    scores = np.dot(vecs_lw.T, qvecs_lw)

    # Sort the Scores From Largest to Smallest as Higher Score Means More Similarity
    ranks = np.expand_dims(np.argsort(-scores, axis=0), axis=1)


    # Take the Top N Ranked Images

    ranks = ranks[:num_returned_images, :]


    final_img_list = []
    query_images = []

    # Go Through Each Query
    for query in range(len(qimages)):
      # Get The Actual Image
      query_image = ImagesFromList(root='', images=[qimages[query]], 
                              imsize=image_size, 
                              bbxs=[self.bbxs[idx]], transform=transforms.ToTensor())[0]
      # Add it to the list of Query Images
      query_images.append(query_image)    
      # Get The Column of Similarity Scores Corresponding to the Current Query
      returned_img_idxs = ranks[:, query]
      retrieved_imgs_concat_list = []
      for idx in returned_img_idxs:
        # Use the Index of the Images to get the The Filename of the images
        img = images[idx]
        retrieved_imgs_concat_list.append(img)
      # Convert a List of Filenames into a List of Retrieved Images

      retrieved_imgs_concat_list = ImagesFromList(root='', images=retrieved_imgs_concat_list, 
                              imsize=image_size, 
                              bbxs=None, transform=transforms.ToTensor())
      # Add the List of Images
      final_img_list.append([retrieved_imgs_concat_list[x].squeeze(0) for x in range(len(retrieved_imgs_concat_list))])

    # To "squeeze"
    query_images = query_images[0]

    return query_images, final_img_list
















  

  


    



# Main Image Retrieval Engine Struct which returns a set of N images for a query image

In [ ]:
import matplotlib.pyplot as plt


class ImageRetrievalSystem():
  def __init__(self, feature_extractor, dbimages, img_size, vecs_lw=None):
    self.model = feature_extractor
    self.normalize = transforms.Normalize(mean=self.model.meta["mean"], std=self.model.meta["std"])
    self.transform = transforms.Compose([transforms.ToTensor(), self.normalize])
    

    if "Lw" in self.model.meta:
      self.Lw = self.model.meta["Lw"]["retrieval-SfM-120k"]["ss"]
    else:
      self.Lw = None
    self.dbimages = dbimages
    self.img_size = img_size


    if (vecs_lw is None):
      vecs = extract_vectors(self.model, self.dbimages, self.img_size, self.transform)
      vecs = vecs.numpy()
      self.vecs_lw = whitenapply(vecs, self.Lw["m"], self.Lw["P"])
    else:
      self.vecs_lw = vecs_lw


  def retrieve_images(self, qimage, numreturnedimgs):
    self.model.cuda()
    self.model.eval()
    qvec = extract_ss(self.model, self.normalize(qimage.cuda()).unsqueeze(0)).cuda()

   
    qvec = np.expand_dims(qvec.cpu().detach().numpy(), 1)
 

    self.qvec_lw = whitenapply(qvec, self.Lw["m"], self.Lw["P"])
    # # Take the Dot Product of the Query Features and Database Features to get a Num_Features x Num_Queries Matrix of Similarities
    scores = np.dot(self.vecs_lw.T, self.qvec_lw)

    # # Sort the Scores From Largest to Smallest as Higher Score Means More Similarity
    full_ranks = np.expand_dims(np.argsort(-scores, axis=0), axis=1)


    # # Take the Top N Ranked Images
    ranks = full_ranks[:numreturnedimgs, :]


    final_img_list = []
  
    # Get The Column of Similarity Scores Corresponding to the Current Query
    returned_img_idxs = ranks[:, 0].squeeze()
    retrieved_imgs_concat_list = []
    for idx in returned_img_idxs:
      # Use the Index of the Images to get the The Filename of the images
      img = self.dbimages[idx]
      retrieved_imgs_concat_list.append(img)

    # Convert a List of Filenames into a List of Retrieved Images
    retrieved_imgs_concat_list = ImagesFromList(root='', images=retrieved_imgs_concat_list, 
                            imsize=self.img_size, 
                            bbxs=None, transform=transforms.ToTensor())
    # Add the List of Images
    final_img_list.append([retrieved_imgs_concat_list[x].unsqueeze(0) for x in range(len(retrieved_imgs_concat_list))])

    # self.visualize_IR_results(qimage, final_img_list)

    return final_img_list, full_ranks,







# Set-Up Feature Extractor

In [ ]:
model_to_pretrained_checkpoint = {
    "VGEM" : "/content/UAP_retrieval/models/retrievalSfM120k-vgg16-gem.pth",
    "RGEM" : "/content/UAP_retrieval/models/retrievalSfM120k-resnet101-gem.pth",
    "AGEM" : "/content/UAP_retrieval/models/retrievalSfM120k-alexnet-gem.pth",
    "AMAC" : "/content/UAP_retrieval/models/retrievalSfM120k-alexnet-mac.pth",
    "RMAC" : "/content/UAP_retrieval/models/retrievalSfM120k-resnet101-mac.pth",
    "VMAC" : "/content/UAP_retrieval/models/retrievalSfM120k-vgg16-mac.pth"
}

# Initialize Feature Extractor
def create_feature_extractor(modelName="VGEM"):
    state = {}
    # Load State Dict
    if modelName in model_to_pretrained_checkpoint.keys():
      state = torch.load(model_to_pretrained_checkpoint[modelName])
      print("Initializing %s Image Retrieval Network..." % (modelName))
    else:
      state = torch.load(model_to_pretrained_checkpoint["VGEM"])
      print("Initializing Default VGG-16 with GEM Whitening Image Retrieval Network...")

    # Initialize Network with Whitening and Pooling
    model = init_network(model= state["meta"]["architecture"],
            pooling= state["meta"]["pooling"],
            whitening= state["meta"]["whitening"],
            mean= state["meta"]["mean"],
            std= state["meta"]["std"],
            pretrained=False,
        )
    model.load_state_dict(state["state_dict"])
    model.meta["Lw"] = state["meta"]["Lw"]

    # CUDA GPU
    model.cuda()
    return model

In [ ]:
# Create Feature Extractors
vgem = create_feature_extractor("VGEM")
vmac = create_feature_extractor("VMAC")
agem = create_feature_extractor("AGEM")
amac = create_feature_extractor("AMAC")
rgem = create_feature_extractor("RGEM")
rmac = create_feature_extractor("RMAC")

Initializing VGEM Image Retrieval Network...


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Initializing VMAC Image Retrieval Network...
Initializing AGEM Image Retrieval Network...
Initializing AMAC Image Retrieval Network...
Initializing RGEM Image Retrieval Network...
Initializing RMAC Image Retrieval Network...


# Configure Dataset/Images

In [ ]:
# Grab Ground Truths Based on Dataset (Paris6k/Oxford5k)

cfg = configdataset("oxford5k", os.path.join(get_data_root(), "test"))
dbimages = [cfg["im_fname"](cfg, i) for i in range(cfg["n"])]
print(dbimages)

['/content/UAP_retrieval/data/test/oxford5k/jpg/ashmolean_000283.jpg', '/content/UAP_retrieval/data/test/oxford5k/jpg/oxford_002124.jpg', '/content/UAP_retrieval/data/test/oxford5k/jpg/radcliffe_camera_000158.jpg', '/content/UAP_retrieval/data/test/oxford5k/jpg/oxford_002084.jpg', '/content/UAP_retrieval/data/test/oxford5k/jpg/oxford_003332.jpg', '/content/UAP_retrieval/data/test/oxford5k/jpg/jesus_000166.jpg', '/content/UAP_retrieval/data/test/oxford5k/jpg/oxford_002241.jpg', '/content/UAP_retrieval/data/test/oxford5k/jpg/radcliffe_camera_000342.jpg', '/content/UAP_retrieval/data/test/oxford5k/jpg/oxford_001431.jpg', '/content/UAP_retrieval/data/test/oxford5k/jpg/all_souls_000110.jpg', '/content/UAP_retrieval/data/test/oxford5k/jpg/oxford_002729.jpg', '/content/UAP_retrieval/data/test/oxford5k/jpg/oxford_002075.jpg', '/content/UAP_retrieval/data/test/oxford5k/jpg/new_001014.jpg', '/content/UAP_retrieval/data/test/oxford5k/jpg/oriel_000001.jpg', '/content/UAP_retrieval/data/test/oxford

# Initialize Image Retrieval Models

In [ ]:
# Initialize IR System Models
ir_system_vgem = ImageRetrievalSystem(vgem, dbimages, 1024)
ir_system_rgem = ImageRetrievalSystem(rgem, dbimages, 1024)
ir_system_agem = ImageRetrievalSystem(agem, dbimages, 1024)
ir_system_vmac = ImageRetrievalSystem(vmac, dbimages, 1024)
ir_system_rmac = ImageRetrievalSystem(rmac, dbimages, 1024)
ir_system_amac = ImageRetrievalSystem(amac, dbimages, 1024)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>>>> 5063/5063 done...


In [ ]:
# Dataset to Iterate over Query Images
dataset = BlackBoxAttackDataset(vgem, 1024, 10, "oxford5k", ir_system_vgem.vecs_lw)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>>>> 55/55 done...


# Visualizations for IR Results

In [ ]:
# Create Query Image => 10 Returned Images Visualizations
def visualize_IR_results( query_image, retrieved_images, filename, transpose_order = (1, 2, 0)):
    fig3 = plt.figure(constrained_layout=True)
    gs = fig3.add_gridspec(6, 7)
    f3_ax1 = fig3.add_subplot(gs[:, :5])
    f3_ax1.axis("off")
    query_image = query_image.squeeze(0)
    f3_ax1.imshow(query_image.detach().cpu().numpy().transpose(transpose_order))
    for i in range(len(retrieved_images[0])):
      if i % 2 == 0:
        f3_ax = fig3.add_subplot(gs[int(i / 2), 5])
        f3_ax.axis("off")
        f3_ax.imshow(retrieved_images[0][i].detach().squeeze(0).cpu().numpy().transpose(transpose_order))
      else:
        f3_ax = fig3.add_subplot(gs[int(i / 2), 6])
        f3_ax.axis("off")
        f3_ax.imshow(retrieved_images[0][i].detach().squeeze(0).cpu().numpy().transpose(transpose_order))
    
    fig3.savefig(filename + ".jpg")

# ZQBA Attack

# Initialize Surrogate Models

In [ ]:
from torch.nn.modules.pooling import AdaptiveAvgPool2d

# Initialize Surrogate Models

device = torch.device("cuda")
googleNet = torchvision.models.googlenet(pretrained=True)
features = list(googleNet.children())[:-1]
features.append(nn.AdaptiveAvgPool2d(output_size=(1, 1)))
googleNet = nn.Sequential(*features)
googleNet.eval()
googleNet = googleNet.to(device)
googleNetPreprocess = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
for p in googleNet.parameters():
  p.requires_grad = False


swin_t = torchvision.models.swin_t()
swin_t.head = nn.ReLU(inplace=True)
swin_t.eval()
swin_t = swin_t.to(device)
swin_t_preprocess = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
for p in swin_t.parameters():
  p.requires_grad = False

regNet = torchvision.models.regnet_x_3_2gf()
regNet = torchvision.models.swin_t()
regNet.fc = nn.ReLU(inplace=True)
regNet.eval()
regNet = regNet.to(device)
regNetPreprocess = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
for p in regNet.parameters():
  p.requires_grad = False



mnasNet = torchvision.models.mnasnet1_3()
mnasNet.classifier = nn.ReLU(inplace=True)
mnasNet.eval()
mnasNet = mnasNet.to(device)
mnasPreprocess = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
for p in mnasNet.parameters():
  p.requires_grad = False

mobileNet = torchvision.models.mobilenet_v3_large()
mobileNet.classifier = nn.ReLU(inplace=True)
mobileNet.eval()
mobileNet = mobileNet.to(device)
mobilePreprocess = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
for p in mobileNet.parameters():
  p.requires_grad = False


squeezeNet = torchvision.models.squeezenet1_1()
squeezeNet.classifier = nn.Sequential(*[nn.AdaptiveAvgPool2d(output_size=(1, 1)), nn.ReLU(inplace=True) ])
squeezeNet.eval()
squeezeNet = mnasNet.to(device)
squeezePreprocess = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
for p in squeezeNet.parameters():
  p.requires_grad = False



denseNet = torchvision.models.densenet121(pretrained=True)
denseNet.classifier = nn.ReLU(inplace=True)
denseNet.eval()
denseNet = denseNet.to(device)
denseNetPreprocess = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
for p in denseNet.parameters():
  p.requires_grad = False

efficientNet = torchvision.models.efficientnet_v2_s(pretrained=True)
efficientNet.classifier = nn.ReLU(inplace=True)
efficientNet.eval()
efficientNet = efficientNet.to(device)
efficientNetPreprocess = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])])
for p in efficientNet.parameters():
  p.requires_grad = False

inceptionV3 = torchvision.models.inception_v3(pretrained=True)
inceptionV3.fc = nn.ReLU(inplace=True)
inceptionV3.eval()
inceptionV3 = inceptionV3.to(device)
inceptionV3Preprocess = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])])
for p in inceptionV3.parameters():
  p.requires_grad = False

convNext = torchvision.models.convnext_small(pretrained=True)
convNext.classifier = nn.ReLU(inplace=True)
convNext.eval()
convNext = convNext.to(device)
convNextPreprocess = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])])
for p in convNext.parameters():
  p.requires_grad = False




/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


  0%|          | 0.00/49.7M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth


  0%|          | 0.00/82.7M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ConvNeXt_Small_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/convnext_small-0c510722.pth" to /root/.cache/torch/hub/checkpoints/convnext_small-0c510722.pth


  0%|          | 0.00/192M [00:00<?, ?B/s]

# NDCG and Relevance Based Loss Function from QAIR

In [ ]:
import math


# QAIR NDCD Using 1.2 as Base Exponent rather than 2 and multiplying by 10
def ndcg(img_list):
  sum_exponential = 0
  probability_list = []
  for i in range(1, len(img_list) + 1):
    relevance = len(img_list) + 1 - i
    exponential = math.pow(1.2, relevance) - 1
    sum_exponential += exponential
    probability_list.append(exponential)
  probability_tensor = torch.tensor(probability_list)
  probability_tensor /= sum_exponential
  probability_tensor *= 10
  return probability_tensor



# Calculates if Image Tensor in List of Image Tensors
def containsImage(listOfImages, image):
  for i in listOfImages:
    if (torch.equal(i, image)):
      return True
  return False


# Calculates the intersection of two image tensor lists
def intersectionOfImageLists(img_list1, img_list2):
  intersection_list = []
  for i in range(len(img_list1)):
    for j in range(len(img_list2)):
      if torch.equal(img_list1[i], img_list2[j]):
        intersection_list.append(img_list1[i])
  return intersection_list


# Based of QAIR
def relevanceBasedLossFunction(clean_results, perturbed_results):
  clean_prob = ndcg(clean_results)
  perturbed_prob = ndcg(perturbed_results)
  print(perturbed_prob)

  img_intersection = intersectionOfImageLists(clean_results, perturbed_results) 
  print(len(img_intersection)) 
  attack_failure_probability = torch.zeros(clean_prob.size())

  for i in range(len(perturbed_results)):
    if containsImage(img_intersection, perturbed_results[i]):
      attack_failure_probability[i] = perturbed_prob[i]
  

  loss = torch.dot(clean_prob, attack_failure_probability)

  return loss












# Main Train Loop

In [ ]:
import pickle
from torch.optim import Adam
import torch

# Initialize Surrogate Models
models = [googleNet, swin_t, squeezeNet, denseNet, mobileNet, mnasNet, convNext, regNet]

# Experiment Constants
NUM_ITERS = 20
NUM_META_TRAIN_STEPS = 8
preprocess = denseNetPreprocess
target = torch.zeros((3, 1000, 1000)).to(device)
LAMBDA = 1.0
EPSILON = 0.4
NUM_QUERY_IMAGES = 55
LR = 0.00625


# Metrics Dictionary
metrics = {}
for model_name in model_to_pretrained_checkpoint.keys():
  metrics[model_name] = {"Number Successful 5 out of 10": 0.0, "Number Successful Top 10" : 0.0, "Number Successful Top 5" : 0.0, "Total Relevance Based Loss" : 0.0, "Full Clean Ranks" : torch.zeros((5063, 55)), "Full Dirty Ranks" : torch.zeros((5063, 55))}


# Method to Update Metrics
def updateMetrics(clean_img_list, full_clean_ranks, dirty_img_list, full_dirty_ranks, model_name, idx):
  metrics[model_name]["Total Relevance Based Loss"] += relevanceBasedLossFunction(clean_img_list[0], dirty_img_list[0])
  print(model_name)
  metrics[model_name]["Full Dirty Ranks"][:, idx] = torch.from_numpy(full_dirty_ranks.squeeze())
  metrics[model_name]["Full Clean Ranks"][:, idx] = torch.from_numpy(full_clean_ranks.squeeze())
  
  if len(np.intersect1d(full_clean_ranks[:5].squeeze(), full_dirty_ranks[:5].squeeze())) == 0:
    metrics[model_name]["Number Successful Top 5"] += 1
  if len(intersectionOfImageLists(clean_img_list[0], dirty_img_list[0])) <= 5:
    metrics[model_name]["Number Successful 5 out of 10"] += 1
  if len(intersectionOfImageLists(clean_img_list[0], dirty_img_list[0])) == 0:
    metrics[model_name]["Number Successful Top 10"] += 1





for i in range(NUM_QUERY_IMAGES):

  # Initialize with original query Image
  img = dataset[i][0].cuda()
  img = Variable(img, requires_grad=True)
  optimizer = Adam([img], lr=LR)

  # Total Iterations
  for iteration in range(NUM_ITERS):
    img.requires_grad_(True)

    # Set Aside MOdels for Ensemble Optimization and Ensemble Refinement
    train_index = random.sample(range(len(models)), 6)
    test_index = train_index.pop()

    # Ensemble Optimization Iterations
    for meta_train_step in range(NUM_META_TRAIN_STEPS):
        # Start Total Loss at Zero
        total_loss = 0
        for idx in train_index:
          # Get Feature Vectors for Adversarial Image and Target Image
          adv_features = models[idx](preprocess(img).unsqueeze(0))
          target_features = models[idx](preprocess(target).unsqueeze(0))

          # Get MSE between Feature Vectors
          embedding_mse = F.mse_loss(adv_features, target_features)
          d =  torch.sum((dataset[i][0].cuda() - img) ** 2)
          
          # Store Loss
          loss = LAMBDA * embedding_mse

          # Fuse Loss Across n modesls
          total_loss += (1 / len(train_index)) * loss

        # BackProp Loss
        img.retain_grad()
        total_loss.backward(retain_graph=True)

        # Optimizer Steps
        optimizer.step()
        optimizer.zero_grad()

    # Get Feature Vectors for Adversarial Image and Target Image for Held-Out Model
    adv_features = models[test_index](preprocess(img).unsqueeze(0))
    target_features = models[test_index](preprocess(target).unsqueeze(0))

    # Calculate MSE
    embedding_mse =  F.mse_loss(adv_features, target_features)

    d =  torch.sum((dataset[i][0].cuda() - img) ** 2)

    # Calculate Loss
    loss = LAMBDA * embedding_mse


    # Backpropagate
    loss.backward(retain_graph=True)
    optimizer.step()

    # Escape if Image Distance greater than 15000
    if (d > 15000):
      break
    a = transforms.ToPILImage() (torch.clamp((img), 0, 1))
    a.save(str(iteration) + "Adam.jpg")

  pert = img - dataset[i][0].cuda()

  # Scale by 2 due to less computational resources
  pert = torch.clamp(pert * 2, -EPSILON, EPSILON)
  new_adv_image = torch.clamp(dataset[i][0].cuda() + pert, 0, 1)

  # Check IR Results across all 6 models
  clean_img_list, full_clean_ranks_vgem = ir_system_vgem.retrieve_images(torch.clamp(dataset[i][0], 0, 1), 10)
  dirty_img_list, full_dirty_ranks_vgem = ir_system_vgem.retrieve_images(torch.clamp(new_adv_image, 0, 1), 10)
  print("VGEM", relevanceBasedLossFunction(clean_img_list[0], dirty_img_list[0]))
  updateMetrics(clean_img_list, full_clean_ranks_vgem, dirty_img_list, full_dirty_ranks_vgem, "VGEM", i)
  visualize_IR_results(new_adv_image, dirty_img_list, "/content/drive/MyDrive/AdversarialAttackResearch/VGEM/dirtyVGEMNew" + str(i))

  clean_img_list, full_clean_ranks_vmac = ir_system_vmac.retrieve_images(torch.clamp(dataset[i][0], 0, 1), 10)
  dirty_img_list, full_dirty_ranks_vmac = ir_system_vmac.retrieve_images(torch.clamp(new_adv_image, 0, 1), 10)
  print("VMAC", relevanceBasedLossFunction(clean_img_list[0], dirty_img_list[0]))
  visualize_IR_results(new_adv_image, dirty_img_list, "/content/drive/MyDrive/AdversarialAttackResearch/VMAC/dirtyVMACNew" + str(i))
  updateMetrics(clean_img_list, full_clean_ranks_vmac, dirty_img_list, full_dirty_ranks_vmac, "VMAC", i)



  clean_img_list, full_clean_ranks_agem = ir_system_agem.retrieve_images(torch.clamp(dataset[i][0], 0, 1), 10)
  dirty_img_list, full_dirty_ranks_agem  = ir_system_agem.retrieve_images(torch.clamp(new_adv_image, 0, 1), 10)
  print("AGEM", relevanceBasedLossFunction(clean_img_list[0], dirty_img_list[0]))
  visualize_IR_results(new_adv_image, dirty_img_list, "/content/drive/MyDrive/AdversarialAttackResearch/AGEM/dirtyAGEMNew" + str(i))
  updateMetrics(clean_img_list, full_clean_ranks_agem, dirty_img_list, full_dirty_ranks_agem, "AGEM", i)



  clean_img_list, full_clean_ranks_amac = ir_system_amac.retrieve_images(torch.clamp(dataset[i][0], 0, 1), 10)
  dirty_img_list, full_dirty_ranks_amac  = ir_system_amac.retrieve_images(torch.clamp(new_adv_image, 0, 1), 10)
  print("AMAC", relevanceBasedLossFunction(clean_img_list[0], dirty_img_list[0]))
  visualize_IR_results(new_adv_image, dirty_img_list, "/content/drive/MyDrive/AdversarialAttackResearch/AMAC/dirtyAMACNew" + str(i))
  updateMetrics(clean_img_list, full_clean_ranks_amac, dirty_img_list, full_dirty_ranks_amac, "AMAC", i)


  clean_img_list, full_clean_ranks_rgem  = ir_system_rgem.retrieve_images(torch.clamp(dataset[i][0], 0, 1), 10)
  dirty_img_list, full_dirty_ranks_rgem = ir_system_rgem.retrieve_images(torch.clamp(new_adv_image, 0, 1), 10)
  print("RGEM", relevanceBasedLossFunction(clean_img_list[0], dirty_img_list[0]))
  visualize_IR_results(new_adv_image, dirty_img_list, "/content/drive/MyDrive/AdversarialAttackResearch/RGEM/dirtyRGEMNew" + str(i))
  updateMetrics(clean_img_list, full_clean_ranks_rgem, dirty_img_list, full_dirty_ranks_rgem, "RGEM", i)



  clean_img_list, full_clean_ranks_rmac = ir_system_rmac.retrieve_images(torch.clamp(dataset[i][0], 0, 1), 10)
  dirty_img_list, full_dirty_ranks_rmac = ir_system_rmac.retrieve_images(torch.clamp(new_adv_image, 0, 1), 10)
  print("RMAC", relevanceBasedLossFunction(clean_img_list[0], dirty_img_list[0]))
  visualize_IR_results(new_adv_image, dirty_img_list, "/content/drive/MyDrive/AdversarialAttackResearch/RMAC/dirtyRMACNew" + str(i))
  updateMetrics(clean_img_list, full_clean_ranks_rmac, dirty_img_list, full_dirty_ranks_rmac, "RMAC", i)


  visualize_IR_results(dataset[i][0], clean_img_list, "/content/drive/MyDrive/AdversarialAttackResearch/Clean/cleanNew" + str(i))

  full_img = transforms.ToPILImage() (new_adv_image)
  full_img.save("/content/drive/MyDrive/AdversarialAttackResearch/Full Images/imgNew" + str(i) + ".jpg")


  # Dump Metrics
  with open('/content/drive/MyDrive/AdversarialAttackResearch/results_progress_new.pkl', 'wb') as f:
    pickle.dump(metrics, f)













Image 0
tensor(423.9276, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1270.4910, device='cuda:0', grad_fn=<SumBackward0>)
tensor(2190.3318, device='cuda:0', grad_fn=<SumBackward0>)
tensor(3224.8279, device='cuda:0', grad_fn=<SumBackward0>)
tensor(4083.2607, device='cuda:0', grad_fn=<SumBackward0>)
tensor(4941.0732, device='cuda:0', grad_fn=<SumBackward0>)
tensor(5601.6577, device='cuda:0', grad_fn=<SumBackward0>)
tensor(6104.2217, device='cuda:0', grad_fn=<SumBackward0>)
tensor(6863.9873, device='cuda:0', grad_fn=<SumBackward0>)
tensor(7545.8115, device='cuda:0', grad_fn=<SumBackward0>)
tensor(8150.2061, device='cuda:0', grad_fn=<SumBackward0>)
tensor(8614.6826, device='cuda:0', grad_fn=<SumBackward0>)
tensor(9016.8369, device='cuda:0', grad_fn=<SumBackward0>)
tensor(9617.2090, device='cuda:0', grad_fn=<SumBackward0>)
tensor(10498.4707, device='cuda:0', grad_fn=<SumBackward0>)
tensor(12627.8848, device='cuda:0', grad_fn=<SumBackward0>)
tensor(14680.8545, device='cuda:0', grad_fn=<Su

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


tensor([2.4547, 1.9668, 1.5602, 1.2213, 0.9390, 0.7037, 0.5076, 0.3442, 0.2080,
        0.0946])
1
VMAC tensor(0.1185)
tensor([2.4547, 1.9668, 1.5602, 1.2213, 0.9390, 0.7037, 0.5076, 0.3442, 0.2080,
        0.0946])
1
VMAC
tensor([2.4547, 1.9668, 1.5602, 1.2213, 0.9390, 0.7037, 0.5076, 0.3442, 0.2080,
        0.0946])
0
AGEM tensor(0.)
tensor([2.4547, 1.9668, 1.5602, 1.2213, 0.9390, 0.7037, 0.5076, 0.3442, 0.2080,
        0.0946])
0
AGEM
tensor([2.4547, 1.9668, 1.5602, 1.2213, 0.9390, 0.7037, 0.5076, 0.3442, 0.2080,
        0.0946])
0
AMAC tensor(0.)
tensor([2.4547, 1.9668, 1.5602, 1.2213, 0.9390, 0.7037, 0.5076, 0.3442, 0.2080,
        0.0946])
0
AMAC
tensor([2.4547, 1.9668, 1.5602, 1.2213, 0.9390, 0.7037, 0.5076, 0.3442, 0.2080,
        0.0946])
1
RGEM tensor(1.4917)
tensor([2.4547, 1.9668, 1.5602, 1.2213, 0.9390, 0.7037, 0.5076, 0.3442, 0.2080,
        0.0946])
1
RGEM
tensor([2.4547, 1.9668, 1.5602, 1.2213, 0.9390, 0.7037, 0.5076, 0.3442, 0.2080,
        0.0946])
1
RMAC tensor(3.868

KeyboardInterrupt: ignored

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7fde85b997a0> (for post_execute):


KeyboardInterrupt: ignored